In [31]:
from joblib import load
import numpy as np
import pandas as pd

import onnxruntime as rt

In [32]:
fea_dict={
    'all':[
"fft_imag_3",
"fft_angle_3",
"alt_stderr_min_50",
"alt_rvalue_min_50",
"fft_real_2",
"fft_real_5",
"erbc_num10_focus2",
"fft_imag_6",
"lumpiness"
],


 'opi':[
"fft_imag_3",
"erbc_num10_focus4",
"alt_rvalue_min_50",
"fft_angle_3",
"erbc_num10_focus2",
"fft_real_5"
],

 'cns':[
"fft_imag_3",
"fft_angle_3",
"fft_real_5",
"minindex",
"lumpiness",
"fft_imag_6",
"alt_rvalue_min_50",
"fft_angle_2",
"erbc_num10_focus2",
"S_down",
"retindex",
"erbc_num10_focus4",
"mac",
"alt_stderr_min_50",
],

'other':[
  "fft_imag_3",
  "fft_angle_3",
  "alt_stderr_min_50",
  "fft_imag_6",
  "fft_real_2",
  "alt_rvalue_min_50",
  "fft_real_5",
  "minvalue",
  "V2",
  "F2",
  "retindex"
]
}
def read_data(tit):
    path = 'E:\\desktop\\data\\app\\223\\fea\\ok\\'
    #读取数据
    data = pd.read_csv(path+tit,encoding='gbk')
    data = data.set_index('name')
    return data

def ooo(way,count_array):
    pred_onx = eval(way+"_mod").run(
        [eval(way+"_mod").get_outputs()[0].name],
        {eval(way+"_mod").get_inputs()[0].name: count_array.astype(np.float32)})[0]

    return pred_onx




all_mod=rt.InferenceSession("./mod/all.onnx")
opi_mod=rt.InferenceSession("./mod/opi.onnx")
cns_mod=rt.InferenceSession("./mod/cns.onnx")
other_mod=rt.InferenceSession("./mod/other.onnx")

##### 多种方式

In [33]:
tit='n.csv'
# tit='all.csv'


fea_data=read_data(tit)

# count_array=np.array(fea_data[fea_dict['all']],dtype='float32')
# print(count_array)
# input_name = all_mod.get_inputs()[0].name
# label_name = all_mod.get_outputs()[0].name

# pred_onx = all_mod.run(
#     [label_name], {input_name: count_array})[0]

# print(pred_onx)

result={'name':list(fea_data.index),'all':[],'opi':[],'cns':[],'other':[]}
for way in ('all','opi','cns','other'):
    fea_list=fea_dict[way]
    count_array=np.array(fea_data[fea_list])
    y_pred=ooo(way,count_array)
    result[way]=list(y_pred)
    

result=pd.DataFrame(result)
result=result.set_index('name',drop=True)

youla={"(opi or cns or other) and all":[],"(opi or cns) and all":[],"四选二":[]}
for i in range(len(result)):
    row=list(result.iloc[i])
    if sum(row[:3])>=1 and row[0]==1:
        youla["(opi or cns or other) and all"].append(1)
    else:
        youla["(opi or cns or other) and all"].append(0)

    if sum(row[:2])>=1 and row[0]==1:
        youla["(opi or cns) and all"].append(1)
    else:
        youla["(opi or cns) and all"].append(0)

    if sum(row)>=2:
        youla["四选二"].append(1)
    else:
        youla["四选二"].append(0)


youla=pd.DataFrame(youla,index=result.index)
result=pd.concat([result,youla],axis=1)
psum=len(result)
for key in result.columns:
    p=sum(result[key])
    print(key,p,'/',psum,p/psum)

result


all 25 / 521 0.04798464491362764
opi 7 / 521 0.013435700575815739
cns 11 / 521 0.02111324376199616
other 14 / 521 0.026871401151631478
(opi or cns or other) and all 25 / 521 0.04798464491362764
(opi or cns) and all 25 / 521 0.04798464491362764
四选二 9 / 521 0.01727447216890595


,all,opi,cns,other,(opi or cns or other) and all,(opi or cns) and all,四选二
name,,,,,,,
20230110-094216-14,0,0,0,0,0,0,0
20221215-144224-D3,0,0,0,0,0,0,0
20230110-095455-6,0,0,0,0,0,0,0
20230111-100411-5,0,0,0,0,0,0,0
20230109-101502-12,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
20230111-095300-12,0,0,0,0,0,0,0
20220805-155022-郑艳芬,0,0,1,0,0,0,0
20230110-094510-13,0,0,0,0,0,0,0
